<a href="https://colab.research.google.com/github/wataru133/Code/blob/master/Convol_FC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Flat Buffer

In [15]:
# Install
!pip install flatbuffers==1.12.0
import flatbuffers
%cd /content/
!rm -rf flatbuffers*
!curl -L "https://github.com/google/flatbuffers/archive/v1.12.0.zip" -o flatbuffers.zip
!unzip -q flatbuffers.zip
!mv flatbuffers-1.12.0 flatbuffers
%cd flatbuffers
!cmake -G "Unix Makefiles" -DCMAKE_BUILD_TYPE=Release
!make -j 8
!cp flatc /usr/local/bin/
%cd /content/
!rm -rf tensorflow
%tensorflow_version 1.x
!wget https://github.com/tensorflow/tensorflow/archive/v2.4.1.zip
!unzip v2.4.1.zip &> 0
!mv tensorflow-2.4.1/ tensorflow/
!flatc --python --gen-object-api tensorflow/tensorflow/lite/schema/schema_v3.fbs
import sys
# This hackery allows us to import the Python files we've just generated.
sys.path.append("/content/tflite/")
import Model

/content
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   124  100   124    0     0    765      0 --:--:-- --:--:-- --:--:--   765
100 1463k    0 1463k    0     0  2540k      0 --:--:-- --:--:-- --:--:-- 6749k
/content/flatbuffers
-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Looking for strtof_l
-- Looking for strtof_l - found
-- L

# Install Tensorflow

In [16]:
!pip install tensorflow==2.4.0

# Import dependencies

In [17]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import os
import random

# Set seed for experiment reproducibility
seed = 133
np.random.seed(seed)
tf.random.set_seed(seed)

# Define paths to model files
MODELS_DIR = 'models/'
if not os.path.exists(MODELS_DIR):
    os.mkdir(MODELS_DIR)
MODEL_TF = MODELS_DIR + 'model'
MODEL_NO_QUANT_TFLITE = MODELS_DIR + 'model_no_quant.tflite'
MODEL_TFLITE = MODELS_DIR + 'model.tflite'
MODEL_TFLITE_MICRO = MODELS_DIR + 'model.cc'

# CNN model

## Input

In [18]:
# Input shape
input_shape = [1, 5, 5, 1]
x = np.random.randint(1,10, size=input_shape).astype(np.float32) # Change type to float 32 for input of representative_dataset for convert to TFLite 
print(x)


[[[[2.]
   [9.]
   [9.]
   [2.]
   [7.]]

  [[5.]
   [6.]
   [3.]
   [7.]
   [6.]]

  [[7.]
   [7.]
   [9.]
   [9.]
   [5.]]

  [[8.]
   [4.]
   [2.]
   [7.]
   [4.]]

  [[6.]
   [7.]
   [9.]
   [3.]
   [9.]]]]


## Model CNN

In [19]:
# Model with 1 CNN layer kenerl size = 3x3
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(1, (3, 3), input_shape=input_shape[1:], use_bias=True))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(2))

print(model.get_weights())

weight  = []
weight1 = np.random.randint(1,10, size=(3,3,1,1)) # Weight of Convol layer
weight2 = np.random.randint(1,10, size=(1)) # Bias of Convol layer
weight3 = np.random.randint(1,10, size=(9,2)) # Weight of Dense layer
weight4 = np.random.randint(1,10, size=(2,)) # Bias of Convol layer
weight.append(weight1)
weight.append(weight2)
weight.append(weight3)
weight.append(weight4)

model.set_weights(weight)
print(model.get_weights())

# Save model
model.save(MODEL_TF)

[array([[[[ 0.01538676]],

        [[-0.5477104 ]],

        [[-0.34695753]]],


       [[[-0.45251068]],

        [[-0.03081071]],

        [[ 0.09090585]]],


       [[[-0.4984012 ]],

        [[-0.39746004]],

        [[-0.33767614]]]], dtype=float32), array([0.], dtype=float32), array([[ 0.18935436, -0.2766963 ],
       [ 0.45313174,  0.6625735 ],
       [ 0.05022651, -0.09159493],
       [ 0.03842396, -0.21747208],
       [ 0.05905038, -0.5173921 ],
       [ 0.5508906 , -0.40164167],
       [-0.3833571 ,  0.729672  ],
       [ 0.7179212 , -0.5358964 ],
       [ 0.36740094,  0.11368233]], dtype=float32), array([0., 0.], dtype=float32)]
[array([[[[7.]],

        [[3.]],

        [[3.]]],


       [[[3.]],

        [[9.]],

        [[3.]]],


       [[[8.]],

        [[3.]],

        [[9.]]]], dtype=float32), array([6.], dtype=float32), array([[3., 8.],
       [8., 3.],
       [8., 1.],
       [3., 1.],
       [5., 9.],
       [4., 2.],
       [4., 3.],
       [2., 5.],
       [7., 4

### TFLite no quantization model

In [20]:
# Convert the TF model to the TensorFlow Lite format without quantization
converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_TF)
model_no_quant_tflite = converter.convert()

# Save the model to disk
open(MODEL_NO_QUANT_TFLITE, "wb").write(model_no_quant_tflite)

1656

### TFLite quantization model

In [21]:
# Set the optimization flag.
converter.optimizations = [tf.lite.Optimize.DEFAULT]

def representative_dataset():
    # yield([x.reshape(input_shape)])
    yield([x])

# Enforce integer only quantization
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8
converter.representative_dataset = representative_dataset

# Convert the TF model to the TensorFlow Lite format with quantization
model_tflite = converter.convert()

# Save the model to disk
open(MODEL_TFLITE, "wb").write(model_tflite)

2056

## Prediction with 3 models

In [22]:
# predict functions for TFLite models
def predict_tflite(tflite_model, x_test):
  # Prepare the test data
  x_test_ = x_test

  # Initialize the TFLite interpreter
  interpreter = tf.lite.Interpreter(model_content=tflite_model)
  interpreter.allocate_tensors()

  input_details = interpreter.get_input_details()[0]
  # print(input_details)
  output_details = interpreter.get_output_details()[0]
  # print(output_details)

  # If required, quantize the input layer (from float to integer)
  input_scale, input_zero_point = input_details["quantization"]
  if (input_scale, input_zero_point) != (0.0, 0):
    x_test_ = x_test_ / input_scale + input_zero_point
    x_test_ = x_test_.astype(input_details["dtype"])
  
  # Invoke the interpreter
  y_pred = np.empty(output_details["shape"], dtype=output_details["dtype"])
  interpreter.set_tensor(input_details["index"], x_test_)
  interpreter.invoke()
  y_pred = interpreter.get_tensor(output_details["index"])
  # print(interpreter.get_tensor(output_details["index"]))
  
  # If required, dequantized the output layer (from integer to float)
  output_scale, output_zero_point = output_details["quantization"]
  if (output_scale, output_zero_point) != (0.0, 0):
    y_pred = y_pred.astype(np.float32)
    y_pred = (y_pred - output_zero_point) * output_scale

  return y_pred

In [23]:
# Output of TF
y = model.predict(x)
print(y)

# Output of TFL no quantization
y = predict_tflite(model_no_quant_tflite, x)
print(y)

# Output of TFL quantization
y = predict_tflite(model_tflite, x)
print(y)

[[13935. 11107.]]
[[13935. 11107.]]
[[13880.354 11038.706]]


# Vela Compiler

In [24]:
# Install
!pip install ethos-u-vela

     |████████████████████████████████| 304 kB 5.4 MB/s 
     |████████████████████████████████| 6.3 MB 40.4 MB/s 
  Created wheel for ethos-u-vela: filename=ethos_u_vela-3.1.0-cp37-cp37m-linux_x86_64.whl size=408673 sha256=c0a927510d5fa086203cc11a51ea91ea2ff24bc985f7affbe5ee1ca04f3fd8e5
  Stored in directory: /root/.cache/pip/wheels/d1/3e/8a/dbbca737198be7d344a05c405e602c86b1d137037fa5774e39
Successfully built ethos-u-vela
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [25]:
# Compile with no quantization model
!vela models/model_no_quant.tflite
!vela models/model.tflite

 - Input(s), Output and Weight tensors must have quantization parameters
   Op has tensors with missing quantization parameters: serving_default_conv2d_8_input:0, std.constant2_reshape, ;;conv2d_8/bias1
 - Input(s), Output and Weight tensors must have quantization parameters
   Op has tensors with missing quantization parameters: ;;conv2d_8/bias1, tfl.reshape
 - Input(s), Output and Weight tensors must have quantization parameters
   Op has tensors with missing quantization parameters: tfl.reshape, std.constant1_reshape, StatefulPartitionedCall:0

Network summary for model_no_quant
Accelerator configuration               Ethos_U55_256
System configuration                 internal-default
Memory mode                          internal-default
Accelerator clock                                 500 MHz


4 passes fused into 4
2/9 (22.2%) operations falling back to the CPU
Neural network macs                                 0 MACs/batch
Network Tops/s                                    nan T

# Flat Buffer

## Load CNN model

In [26]:
# Function for read/write flat buffer
def load_model_from_file(model_filename):
  with open(model_filename, "rb") as file:
    buffer_data = file.read()
  model_obj = Model.Model.GetRootAsModel(buffer_data, 0)
  model = Model.ModelT.InitFromObj(model_obj)
  return model

def save_model_to_file(model, model_filename):
  builder = flatbuffers.Builder(1024)
  model_offset = model.Pack(builder)
  builder.Finish(model_offset, file_identifier=b'TFL3')
  model_data = builder.Output()
  with open(model_filename, 'wb') as out_file:
    out_file.write(model_data)
    
# Load CNNO model no quantization
model = load_model_from_file(MODEL_NO_QUANT_TFLITE)

## Model information

In [27]:
print(model.version)

print(model.description)

i = 0
for buffer in model.buffers:
  print(f"\nbuffer[{i}] = {buffer.data}")
  if buffer.data is not None: # np.frombuffer dont accepted NoneType
    original_weights = np.frombuffer(buffer.data, dtype=np.float32)
    print(f"weights = {original_weights}\n")
  i+=1

3
b'MLIR Converted.'

buffer[0] = None

buffer[1] = None

buffer[2] = [ 0  0  0 65  0  0 16 65]
weights = [8. 9.]


buffer[3] = [255 255 255 255   9   0   0   0]
weights = [    nan 1.3e-44]


buffer[4] = [  0   0  64  64   0   0   0  65   0   0   0  65   0   0  64  64   0   0
 160  64   0   0 128  64   0   0 128  64   0   0   0  64   0   0 224  64
   0   0   0  65   0   0  64  64   0   0 128  63   0   0 128  63   0   0
  16  65   0   0   0  64   0   0  64  64   0   0 160  64   0   0 128  64]
weights = [3. 8. 8. 3. 5. 4. 4. 2. 7. 8. 3. 1. 1. 9. 2. 3. 5. 4.]


buffer[5] = [  0   0 224  64   0   0  64  64   0   0  64  64   0   0  64  64   0   0
  16  65   0   0  64  64   0   0   0  65   0   0  64  64   0   0  16  65]
weights = [7. 3. 3. 3. 9. 3. 8. 3. 9.]


buffer[6] = [  0   0 192  64]
weights = [6.]


buffer[7] = None

buffer[8] = None

buffer[9] = None

buffer[10] = [49 46 53 46 48  0  0  0  0  0  0  0  0  0  0  0]
weights = [4.119566e-11 6.726233e-44 0.000000e+00 0.000000e+00]



In [28]:
# Model has 1 subgraph
print(model.subgraphs[0].name) # Name "Main"
print(model.subgraphs[0].inputs) # Input has index 0
print(model.subgraphs[0].outputs) # Output has index 3

b'main'
[0]
[8]


In [29]:
## Subgraph 0 has 1 operator
print(model.subgraphs[0].operators[0].opcodeIndex) # Operation information store in index 0 of model.operatorCodes
print(model.operatorCodes[0].builtinCode) # Convol 2D
print(model.operatorCodes[0].customCode) # No custom code

print(model.subgraphs[0].operators[0].inputs)

print(model.subgraphs[0].operators[0].outputs)

print(model.subgraphs[0].operators[0].builtinOptions) # Conv2DOptions
print(model.subgraphs[0].operators[0].builtinOptions.padding) # VALID
print(model.subgraphs[0].operators[0].builtinOptions.strideW) # 1
print(model.subgraphs[0].operators[0].builtinOptions.strideH) # 1
print(model.subgraphs[0].operators[0].builtinOptions.fusedActivationFunction) # NONE

print(model.subgraphs[0].operators[0].customOptions) # No custom option 

0
3
None
[0 4 5]
[6]
1
1
1
0
None


In [30]:
for index, tensor in enumerate(model.subgraphs[0].tensors):
  print(f"tensor[{index}].name         = {tensor.name}") 
  print(f"tensor[{index}].shape        = {tensor.shape}")
  print(f"tensor[{index}].type         = {tensor.type}") # 0: FLOAT3
  print(f"tensor[{index}].buffer_index = {tensor.buffer}") 
  print(f"tensor[{index}].quantization[min, max, scale, zeroPoint] = [{tensor.quantization.min}, {tensor.quantization.max}, {tensor.quantization.scale}, {tensor.quantization.zeroPoint}]\n\n") 

tensor[0].name         = b'serving_default_conv2d_8_input:0'
tensor[0].shape        = [1 5 5 1]
tensor[0].type         = 0
tensor[0].buffer_index = 1
tensor[0].quantization[min, max, scale, zeroPoint] = [None, None, None, None]


tensor[1].name         = b'dense_7/bias'
tensor[1].shape        = [2]
tensor[1].type         = 0
tensor[1].buffer_index = 2
tensor[1].quantization[min, max, scale, zeroPoint] = [None, None, None, None]


tensor[2].name         = b'std.constant'
tensor[2].shape        = [2]
tensor[2].type         = 2
tensor[2].buffer_index = 3
tensor[2].quantization[min, max, scale, zeroPoint] = [None, None, None, None]


tensor[3].name         = b'std.constant1'
tensor[3].shape        = [2 9]
tensor[3].type         = 0
tensor[3].buffer_index = 4
tensor[3].quantization[min, max, scale, zeroPoint] = [None, None, None, None]


tensor[4].name         = b'std.constant2'
tensor[4].shape        = [1 3 3 1]
tensor[4].type         = 0
tensor[4].buffer_index = 5
tensor[4].quantization[m